# Oscilloscope utility – using Ethernet

In [1]:
import matplotlib.pyplot as plt
import sys
import os
import time
import h5py
import numpy as np
import glob
import vxi11

# Step 0:
# Connect oscilloscope via direct Ethernet link
# Step 1:
# Run this cell to verify the connection.
# Step 2:
# Run next cell to initialize grabbing.


# This will need a rewrite
class TmcDriver:

    def __init__(self, device):
        print("Initializing connection to: " + device)
        self.device = device
        self.instr = vxi11.Instrument(device)
 
    def write(self, command):
        #print command
        self.instr.write(command);

    def read(self, length = 500):
        return self.instr.read(length)

    def read_raw(self, length = 500):
        return self.instr.read_raw(length)
 
    def getName(self):
        self.write("*IDN?")
        return self.read(300)
    
    def ask(self, command):
        return self.instr.ask(command)
 
    def sendReset(self):
        self.write("*RST")  # Be carefull, this real resets an oscilloscope
        
# For Ethernet
osc = TmcDriver("TCPIP::147.231.24.166::INSTR")
#osc = TmcDriver("TCPIP::10.1.1.3::INSTR")
print(osc.ask("*IDN?"))

Initializing connection to: TCPIP::147.231.24.166::INSTR
RIGOL TECHNOLOGIES,DS2072A,DS2D191200333,00.03.05.SP4


## Read repeatedly records from oscilloscope

In [5]:
filename = 1
    
if (filename == 1):
    for f in glob.iglob("./data/*.h5"): # delete all .h5 files 
        print 'Deleting', f
        os.remove(f)
else:
    print 'Not removing old files, as filename {0} is not 1.'.format(filename)


osc.write(':STOP') # go to STOP mode
time.sleep(0.5)

start_wfd = 0.005 #0.05 #0.012
wfd=start_wfd

while True:
    osc.write(':FUNC:WREC:OPER REC') # start recording
    run_start_time = time.time()
    print '  Capturing...'
    time.sleep(0.5)
    
    while True:
        osc.write(':FUNC:WREC:OPER?') # finish recording?
        reply = osc.read()
        if reply == 'STOP':
            run_time = round(time.time() - run_start_time, 2)
            print('  Subrun finished, capturing for %.2f seconds.' % run_time)
            break
        time.sleep(0.01)

    for channel in ['CHAN1','CHAN2']:
        if (osc.ask(':'+channel+':DISP?') == u'0'):
            print(channel+' is not enabled')
        else:
            print('Reading out '+channel)
            osc.write(':WAV:SOUR '+channel)
            osc.write(':WAV:MODE NORM')
            osc.write(':WAV:FORM BYTE')
            osc.write(':WAV:POIN 1400')

            osc.write(':WAV:XINC?')
            xinc = float(osc.read(100))
            print 'XINC:', xinc,
            osc.write(':WAV:YINC?')
            yinc = float(osc.read(100))
            print 'YINC:', yinc,
            osc.write(':TRIGger:EDGe:LEVel?')
            trig = float(osc.read(100))
            print 'TRIG:', trig,
            osc.write(':WAVeform:YORigin?')
            yorig = float(osc.read(100))
            print 'YORIGIN:', yorig,
            osc.write(':WAVeform:XORigin?')
            xorig = float(osc.read(100))
            print 'XORIGIN:', xorig,
            osc.write(':FUNC:WREP:FEND?') # get number of last frame
            frames = int(osc.read(100))
            print 'FRAMES:', frames, 'SUBRUN', filename

            lastwave = bytearray()

            with h5py.File('./data/data'+'{:02.0f}'.format(filename)+'_'+str(int(round(run_start_time,0)))+'-'+channel+'.h5', 'w') as hf:       
                hf.create_dataset('FRAMES', data=(frames)) # write number of frames
                hf.create_dataset('XINC', data=(xinc)) # write axis parameters
                hf.create_dataset('YINC', data=(yinc))
                hf.create_dataset('TRIG', data=(trig))
                hf.create_dataset('YORIGIN', data=(yorig))
                hf.create_dataset('XORIGIN', data=(xorig))
                hf.create_dataset('CAPTURING', data=(run_time))
                osc.write(':FUNC:WREP:FCUR 1') # go to first frame
                time.sleep(0.5)
                for n in range(1,frames+1):
                    osc.write(':FUNC:WREP:FCUR ' + str(n)) # skip to n-th frame
                    while True:
                        time.sleep(0.05)
                        fcur = osc.ask(':FUNC:WREP:FCUR?')
                        if (str(n) == fcur):
                            # Rigol returns correct current frame
                            sys.stdout.write(str(n))
                            break
                        else:
                            # Rigol has not yet made the seek, wait
                            # or consider extending the sleep above
                            print("Needwait: "+str(n)+' vs '+fcur)

                    reread_count = 0
                    while True:
                        time.sleep(wfd)        
                        osc.write(':WAV:DATA?') # read data
                        time.sleep(wfd)
                        wave1 = bytearray(osc.read_raw(500))
                        wave2 = bytearray(osc.read_raw(500))
                        wave3 = bytearray(osc.read_raw(500))
                        #wave4 = bytearray(osc.read(500))
                        #wave = np.concatenate((wave1[11:],wave2[:(500-489)],wave3[:(700-489)]))
                        wave = np.concatenate((wave1[11:],wave2,wave3[:-1]))
                        if (np.array_equal(wave, lastwave)):
                            wfd = wfd + 0.005
                            print(' Same waveform, wait ' + str(wfd) + ' and reread')
                            reread_count = reread_count + 1
                            if (reread_count > 5):
                                print('------------ Wrong trigger level?')
                        else:
                            hf.create_dataset(str(n), data=wave)
                            lastwave = wave
                            sys.stdout.write('.')
                            wfd = start_wfd
                            break
            print 'OK'
            # End of channel recording
    filename = filename + 1
    

  Capturing...
  Subrun finished, capturing for 2.05 seconds.
Reading out CHAN1
XINC: 2e-10 YINC: 0.0016 TRIG: 0.0352 YORIGIN: -99.0 XORIGIN: -3.96e-08 FRAMES: 508 SUBRUN 1
1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.71.72.73.74.75.76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.93.94.95.96.97.98.99.100.101.102.103.104.105.106.107.108.109.110.111.112.113.114.115.116.117.118.119.120.121.122.123.124.125.126.127.128.129.130.131.132.133.134.135.136.137.138.139.140.141.142.143.144.145.146.147.148.149.150.151.152.153.154.155.156.157.158.159.160.161.162.163.164.165.166.167.168.169.170.171.172.173.174.175.176.177.178.179.180.181.182.183.184.185.186.187.188.189.190.191.192.193.194.195.196.197.198.199.200.201.202.203.204.205.206.207.208.209.210.211.212.213.214.215.216.217.218.219.220.221.222.223.224.225.226.227.228.229.230.231.232.233.234

1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.71.72.73.74.75.76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.93.94.95.96.97.98.99.100.101.102.103.104.105.106.107.108.109.110.111.112.113.114.115.116.117.118.119.120.121.122.123.124.125.126.127.128.129.130.131.132.133.134.135.136.137.138.139.140.141.142.143.144.145.146.147.148.149.150.151.152.153.154.155.156.157.158.159.160.161.162.163.164.165.166.167.168.169.170.171.172.173.174.175.176.177.178.179.180.181.182.183.184.185.186.187.188.189.190.191.192.193.194.195.196.197.198.199.200.201.202.203.204.205.206.207.208.209.210.211.212.213.214.215.216.217.218.219.220.221.222.223.224.225.226.227.228.229.230.231.232.233.234.235.236.237.238.239.240.241.242.243.244.245.246.247.248.249.250.251.252.253.254.255.256.257.258.259.260.261.262.263.264.265.266.267.268.269.270.271.272.273.274.275.276.277.

1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.71.72.73.74.75.76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.93.94.95.96.97.98.99.100.101.102.103.104.105.106.107.108.109.110.111.112.113.114.115.116.117.118.119.120.121.122.123.124.125.126.127.128.129.130.131.132.133.134.135.136.137.138.139.140.141.142.143.144.145.146.147.148.149.150.151.152.153.154.155.156.157.158.159.160.161.162.163.164.165.166.167.168.169.170.171.172.173.174.175.176.177.178.179.180.181.182.183.184.185.186.187.188.189.190.191.192.193.194.195.196.197.198.199.200.201.202.203.204.205.206.207.208.209.210.211.212.213.214.215.216.217.218.219.220.221.222.223.224.225.226.227.228.229.230.231.232.233.234.235.236.237.238.239.240.241.242.243.244.245.246.247.248.249.250.251.252.253.254.255.256.257.258.259.260.261.262.263.264.265.266.267.268.269.270.271.272.273.274.275.276.277.

1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.71.72.73.74.75.76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.93.94.95.96.97.98.99.100.101.102.103.104.105.106.107.108.109.110.111.112.113.114.115.116.117.118.119.120.121.122.123.124.125.126.127.128.129.130.131.132.133.134.135.136.137.138.139.140.141.142.143.144.145.146.147.148.149.150.151.152.153.154.155.156.157.158.159.160.161.162.163.164.165.166.167.168.169.170.171.172.173.174.175.176.177.178.179.180.181.182.183.184.185.186.187.188.189.190.191.192.193.194.195.196.197.198.199.200.201.202.203.204.205.206.207.208.209.210.211.212.213.214.215.216.217.218.219.220.221.222.223.224.225.226.227.228.229.230.231.232.233.234.235.236.237.238.239.240.241.242.243.244.245.246.247.248.249.250.251.252.253.254.255.256.257.258.259.260.261.262.263.264.265.266.267.268.269.270.271.272.273.274.275.276.277.

1.2.3.4.5.6.7.8.9.10.11.12.13.14.15.16.17.18.19.20.21.22.23.24.25.26.27.28.29.30.31.32.33.34.35.36.37.38.39.40.41.42.43.44.45.46.47.48.49.50.51.52.53.54.55.56.57.58.59.60.61.62.63.64.65.66.67.68.69.70.71.72.73.74.75.76.77.78.79.80.81.82.83.84.85.86.87.88.89.90.91.92.93.94.95.96.97.98.99.100.101.102.103.104.105.106.107.108.109.110.111.112.113.114.115.116.117.118.119.120.121.122.123.124.125.126.127.128.129.130.131.132.133.134.135.136.137.138.139.140.141.142.143.144.145.146.147.148.149.150.151.152.153.154.155.156.157.158.159.160.161.162.163.164.165.166.167.168.169.170.171.172.173.174.175.176.177.178.179.180.181.182.183.184.185.186.187.188.189.190.191.192.193.194.195.196.197.198.199.200.201.202.203.204.205.206.207.208.209.210.211.212.213.214.215.216.217.218.219.220.221.222.223.224.225.226.227.228.229.230.231.232.233.234.235.236.237.238.239.240.241.242.243.244.245.246.247.248.249.250.251.252.253.254.255.256.257.258.259.260.261.262.263.264.265.266.267.268.269.270.271.272.273.274.275.276.277.

...

Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167

Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167

Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167

Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167 vs 166
Needwait: 167

KeyboardInterrupt: 